# MS&E 234 Project

### Data cleaning

In [1]:
import pandas as pd
import numpy as np
import re
from random import sample
import itertools
from collections import defaultdict
from tqdm.notebook import tqdm

### Load data

In [1]:
print("test")

test


In [3]:
df_raw = pd.DataFrame()
for i in range(1, 4+1):
    df_raw = pd.concat([df_raw, pd.read_csv(f'../netflix-prize-kaggle-data/combined_data_{i}.txt',
        header=None,
        names=['CustomerID', 'Rating', 'Date'])])

df_raw

,CustomerID,Rating,Date
0,1:,NaN,NaN
1,1488844,3.0,2005-09-06
2,822109,5.0,2005-05-13
3,885013,4.0,2005-10-19
4,30878,4.0,2005-12-26
...,...,...,...
26851921,1790158,4.0,2005-11-01
26851922,1608708,3.0,2005-07-19
26851923,234275,1.0,2004-08-07
26851924,255278,4.0,2004-05-28


### Subset data as in paper (Calandrino 2011)

In [4]:
df = df_raw
print(f'Entire dataset: {df.shape[0]} ratings; {df.dropna().CustomerID.nunique()} users')

# Sample 10000 users
df = df[df['CustomerID'].isin(sample(df.dropna()['CustomerID'].unique().tolist(), 10000))]
print(f"Subset of 10000 users: {df.shape[0]} ratings; {df['CustomerID'].nunique()} users")

# Assume each user makes a random 50% of transactions public
idxs = df[['CustomerID']].reset_index().groupby('CustomerID').agg({'index':lambda x: list(x)}).to_numpy().tolist()
drop_idxs = []
for i in range(len(idxs)):
    drop_idxs.append(sample(idxs[i][0], len(idxs[i][0]) // 2))
drop_idxs = list(itertools.chain(*drop_idxs))
df = df.drop(drop_idxs)
print(f'After making 50% private: {df.shape[0]} ratings; {df.CustomerID.nunique()} users')

# Only consider users with at least 100 public transactions
df = df[df['CustomerID'].isin(df['CustomerID'].value_counts()[df['CustomerID'].value_counts() > 100].index)]
print(f'Filter for users with >= 100 public transactions: {df.shape[0]} ratings; {df.CustomerID.nunique()} users')

# Subset the data to analyze only ratings from July 2005
df = df[df['Date'].apply(lambda x: type(x) == str and bool(re.match(r'2005-07.*', x)))]
print(f'Only July 2005: {df.shape[0]} ratings; {df["CustomerID"].nunique()} users')

Entire dataset: 100498277 ratings; 497959 users
Subset of 10000 users: 2057552 ratings; 10000 users
After making 50% private: 1001625 ratings; 9997 users
Filter for users with >= 100 public transactions: 758877 ratings; 3012 users
Only July 2005: 33704 ratings; 1914 users


In [5]:
df

,CustomerID,Rating,Date
3366,2375677,4.0,2005-07-11
5928,1725035,5.0,2005-07-24
7816,1041575,4.0,2005-07-13
7903,399705,4.0,2005-07-10
16789,2047318,5.0,2005-07-12
...,...,...,...
26830200,2189692,4.0,2005-07-12
26835396,1657693,4.0,2005-07-07
26846405,1170030,1.0,2005-07-10
26849394,1084792,4.0,2005-07-20


### TODO - Mac Comments

In [6]:
# Sanity check: there are ~1570 such users
# Sanity check: there are around 1510 transactions during the period in question 

# TODO there are way more transactions - how do we get only ~1 transaction per user per month, 
# when we are filtering for users that recommend many movies? Seems plausible that these users are 
# making ~20 transactions/month...

# Restrict the attack to (customer, date) pairs in which the customer made 5 or fewer transactions: 
# TODO I think this means only compute prediction accuracy on these pairs, rather than removing them before training

In [7]:
# na's represent the number of movies in the dataset
df_raw.isna().sum()

CustomerID        0
Rating        17770
Date          17770
dtype: int64

In [8]:
# extract movie ID's from raw data, combine these with df
# The data files follow the following format:
# Movie ID:
# CustomerID, Rating, Date
# ...

movieDF = df_raw[df_raw['Rating'].isnull()]

In [9]:
movieRows = np.array(movieDF.index)
movieIDs = []
currIdx = 0
for row in df.itertuples(index = True, name = 'Pandas'):
    currRow = row.Index
    while currIdx < len(movieRows) - 1:
        nextMovieRow = movieRows[currIdx + 1]
        if currRow > nextMovieRow:
            currIdx += 1
        else:
            break
    movieIDs.append(currIdx)

In [10]:
df["MovieID"] = movieIDs
df['Day'] = pd.DatetimeIndex(df['Date']).day

In [12]:
# input: 2 sets x, y
# output: for binary vectors x and y: cosine similarity = |x and y| / sqrt(|x||y|)
def getCosSim(x, y):
    return len(x.intersection(y)) / np.sqrt(len(x) * len(y))

In [13]:
# input: sparse binary adjList
# output: dict of the 50 most similar items and scores, in format: dict[movieID] => ([(items, scores)])
def getSimListDaily(adjList):
    simList = {}
    for key1 in adjList.keys():
        cosSims = []
        for key2 in adjList.keys():
            if key1 == key2:
                continue
            cosSim = getCosSim(adjList[key1], adjList[key2])
            cosSims.append((key2, cosSim))
        simList[key1] = sorted(cosSims, key = lambda x: (-x[1], x[0]))[:50]
    return simList

In [14]:
# input: dataframe of customerIDs, Ratings, Day of Month, and MovieIDs
# input dataframe should contain both public and private data, as the recc system uses all available information
# output: list of 50 related movies and similarity ratings per day
# on each consecutive day, more data is used by the recc system
# uses cosine similarity on a binary matrix
def getSimListMonthly(df):
    # store binary matrix in sparse adjacency list format
    # adjList[movieID] => set of customerIDs
    adjList = defaultdict(set)
    
    simLists = []
    
    for day in tqdm(sorted(df["Day"].unique())):
        # add new movies from today to adjList
        currDF = df[df["Day"] == day]
        for row in currDF.itertuples(index = True, name = 'Pandas'):
            adjList[row.MovieID].add(row.CustomerID)
        
        # compute similarity scores
        currSimList = getSimListDaily(adjList)
        simLists.append(currSimList)
    return simLists

In [15]:
simLists = getSimListMonthly(df)

  0%|          | 0/31 [00:00<?, ?it/s]

In [16]:
simLists[0][12]

KeyError: 12

In [ ]:
simLists[30][12]

### TODO - Viet Comments 2/27

added code to add movieIDs to df

wrote code to generate daily top 50 cosine similarities + scores

regarding Mac's comment: 
"Sanity check: there are around 1510 transactions during the period in question"
"TODO there are way more transactions - how do we get only ~1 transaction per user per month, 
when we are filtering for users that recommend many movies? Seems plausible that these users are making ~20 transactions/month..."

I think the paper made a mistake - there are ~ 35K transactions per month, maybe the 1.5K is referring to daily transactions? I am getting 6.4K (customer, date) pairs.

# for whoever is generating the plots: make sure the train and test sets are properly built.

currently the df generated removes 50% of entries.

When generating the similarity lists, we want to keep 100% of entries.

When making predictions, we get to look at 50% of entries.

In [ ]:
df.groupby(["CustomerID", "Day"]).sum()